In [1]:
import pandas as pd
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import NMF

import pickle

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df=pd.read_pickle("all_recipes.pkl")
df2=pd.read_pickle("all_users.pkl")
df.head()

,calories,category,ingredients,ratings,recipe_id,reviews,title,total_mins,Season,DaytimeOfCooking,Veg/NonVeg
0,728,"['Desserts', 'Fruit Desserts', 'Pineapple Dess...","['2 cups all-purpose flour', '1 teaspoon salt'...",4.703704,263516,23,Chef John's Carrot Cake,95,Fall/Winter,Dessert,Non-Veg (Eggs)
1,692,"['Bread', 'Yeast Bread']","['1/2 cup warm milk', '1/3 cup warm water', '1...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,Any,Lunch/Dinner,Non-Veg (Cheese)
2,209,"['Soups, Stews and Chili', 'Stews', 'Beef']","['1 (1 1/2-pound) flank steak', '2 teaspoons k...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,Any,Dinner,Non-Veg (Beef)
3,505,"['Everyday Cooking', 'Vegetarian', 'Side Dishes']","['2 cups whole milk', '1 teaspoon kosher salt,...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,Any,Lunch/Dinner,Veg
4,291,"['Bread', 'Yeast Bread']","['1 cup warm water', '1 tablespoon white sugar...",5.000000,271049,4,Kouign-Amann,275,Any,Dessert,Non-Veg (Butter)


In [3]:
df2.head()

,date,rating,recipe_id,review,user_id,username
0,2012-09-11,5,222234,This recipe was delicious! Instead of cherry p...,3419993,Pie84
1,2012-12-25,5,222234,Made this recipe as instructed and it was easy...,3154459,hertzen
2,2012-11-22,5,222234,This was a great recipe to make using leftover...,10370475,suziloo
3,2015-07-07,5,222234,"Tasty , simple dessert that comes out cute as ...",2304335,Chelsea M.
4,2016-12-31,5,222234,"Delicious!! I was pressed for time, so I didn'...",1967176,Amanda H.


In [4]:
null_records = df[df.isnull().any(axis=1)]
print(null_records)

    calories                                           category  \
748        0                                                NaN   
753        0                                                NaN   
755        0                                                NaN   
756        0                                                NaN   
774        0                                                NaN   
776      135  ['Appetizers and Snacks', 'Dips and Spreads', ...   
779        0                                                NaN   
784        0                                                NaN   
786        0                                                NaN   
794        0                                                NaN   
804        0                                                NaN   
811        0                                                NaN   
812        0                                                NaN   

                                           ingredients   rati

We Find one record which has a Null Attribute that can be filled manually with relevant data. 

In [5]:
df.at[776, 'Veg/NonVeg'] = 'Non-Veg'

Rest of the Null Attributes are beyond repair. so we drop them from the dataset for better model functionality.

In [6]:
df = df.dropna()
df.drop_duplicates(inplace=True)
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1162 entries, 0 to 1161
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   calories          1162 non-null   object 
 1   category          1162 non-null   object 
 2   ingredients       1162 non-null   object 
 3   ratings           1162 non-null   float64
 4   recipe_id         1162 non-null   int64  
 5   reviews           1162 non-null   object 
 6   title             1162 non-null   object 
 7   total_mins        1162 non-null   int64  
 8   Season            1162 non-null   object 
 9   DaytimeOfCooking  1162 non-null   object 
 10  Veg/NonVeg        1162 non-null   object 
dtypes: float64(1), int64(2), object(8)
memory usage: 100.0+ KB


In [7]:
df2.dropna()
df2.drop_duplicates(inplace=True)
df2=df2.reset_index(drop=True)

def processing(value):
    return int(value)

df2['rating']=df2['rating'].apply(processing)
df2['recipe_id']=df2['recipe_id'].apply(processing)
df2['user_id']=df2['user_id'].apply(processing)

df2['date'] = pd.to_datetime(df2['date'], format='%Y-%m-%d')

**CONVERTING THE FEATURES TO RELEVANT DATATYPE - REVIEWS, CALORIES, CATEGORY, INGREDIENTS**

In [8]:
def replace_k(value):
    if 'k' in value:
        value=str(value).split('k')
        value=value[0]
        value=float(value)*1000
    return value

df['reviews']=df['reviews'].apply(replace_k)

def cat_2(value):
    return eval(value)

df['category']=df['category'].apply(cat_2)
df['ingredients']=df['ingredients'].apply(cat_2)

df['reviews']=df['reviews'].apply(processing)

def calories(value):
    if value.isdigit():
        return int(value)
    else:
        return 1
df['calories']=df['calories'].apply(calories)

df = df.rename(columns={'ratings': 'avg_rating', 'reviews':'no_reviews', 'Season':'season','DaytimeOfCooking':'meal_type','Veg/NonVeg':'diet_type'})

df.head()

,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type
0,728,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95,Fall/Winter,Dessert,Non-Veg (Eggs)
1,692,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,Any,Lunch/Dinner,Non-Veg (Cheese)
2,209,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,Any,Dinner,Non-Veg (Beef)
3,505,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,Any,Lunch/Dinner,Veg
4,291,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275,Any,Dessert,Non-Veg (Butter)


In [9]:
df['season'] = df['season'].apply(lambda x: x.split('/'))
df['season'] = df['season'].apply(lambda x: ['Summer', 'Spring', 'Fall', 'Winter'] if x == ['Any'] else x)
df['meal_type'] = df['meal_type'].apply(lambda x: x.split('/'))

In [10]:
def count_ingredients(ingredients):
    return len(ingredients)

df['no_ingredients'] = df['ingredients'].apply(count_ingredients)
df.head()

,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type,no_ingredients
0,728,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95,"[Fall, Winter]",[Dessert],Non-Veg (Eggs),18
1,692,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",Non-Veg (Cheese),14
2,209,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,"[Summer, Spring, Fall, Winter]",[Dinner],Non-Veg (Beef),24
3,505,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",Veg,10
4,291,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275,"[Summer, Spring, Fall, Winter]",[Dessert],Non-Veg (Butter),10


In [11]:
pattern = re.compile(r'\s*\([^)]*\)')
records = df['diet_type']

cleaned_records = [re.sub(pattern, '', record) for record in records]
df['diet_type'] = cleaned_records

In [12]:
def clean_text(text):
    text = text.lower()
    nopunct_text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = word_tokenize(nopunct_text)
    return tokens[0]

df['diet_type'] = df['diet_type'].apply(clean_text)
df.head()

,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type,no_ingredients
0,728,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95,"[Fall, Winter]",[Dessert],nonveg,18
1,692,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",nonveg,14
2,209,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),690,"[Summer, Spring, Fall, Winter]",[Dinner],nonveg,24
3,505,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",veg,10
4,291,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275,"[Summer, Spring, Fall, Winter]",[Dessert],nonveg,10


In [13]:
# Define a threshold for identifying significant outliers (e.g., 3 standard deviations above the mean)
threshold = 3 * df['total_mins'].std()

# Filter the dataset to include only records with cooking times exceeding the threshold
outliers_df = df[df['total_mins'] > threshold]

# Print the records with significant outlier cooking times
print(outliers_df)

      calories                                           category  \
975        158                            [Seafood, Fish, Salmon]   
980        790               [Main Dish, Pork, Pork Chops, Baked]   
991        484                 [Meat and Poultry, Pork, Shoulder]   
993        316              [Bread, Yeast Bread, Sourdough Bread]   
994         12  [Side Dish, Sauces and Condiments, Canning and...   
997         48  [Appetizers and Snacks, Dips and Spreads, Chee...   
998        120                                                 []   
1039        16                 [Side Dish, Sauces and Condiments]   
1059        52         [Side Dish, Sauces and Condiments, Sauces]   
1148        23                 [Side Dish, Sauces and Condiments]   

                                            ingredients  avg_rating  \
975   [1/4 cup kosher salt, 1/4 cup white sugar, 1 p...    5.000000   
980   [Brine:, 1 1/2 cups boiling water, 3/4 cup pac...    4.600000   
991   [1 (7 pound) fresh, b

In [14]:
df.describe()

,calories,avg_rating,recipe_id,no_reviews,total_mins,no_ingredients
count,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000,1162.000000
mean,375.978485,4.544060,239943.815835,62.666093,194.444062,11.092943
std,295.153220,0.558952,15251.066547,133.848245,771.197728,4.601423
min,1.000000,0.000000,219075.000000,0.000000,0.000000,1.000000
25%,194.000000,4.458333,228643.250000,8.000000,25.000000,8.000000
50%,317.000000,4.659545,237471.500000,22.000000,55.000000,11.000000
75%,489.750000,4.803826,254277.750000,54.000000,140.000000,14.000000
max,4709.000000,5.000000,273495.000000,2000.000000,14415.000000,29.000000


In [15]:
df.describe(include="object").T

,count,unique,top,freq
category,1162,440,"[World Cuisine, European, Italian]",58
ingredients,1162,1156,"[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",2
title,1162,1150,Chef John's Carrot Cake,2
season,1162,9,"[Summer, Spring, Fall, Winter]",883
meal_type,1162,59,[Dinner],445
diet_type,1162,2,veg,593


**OUTLIERS REMOVAL USING IQR METHOD**

IQR is a robust statistical measure that represents the spread of the middle 50% of the data. It's calculated as the difference between the third quartile (Q3) and the first quartile (Q1).
In recipe recommendation, outliers could be recipes with:

    Extremely high or low cooking time.
    Extremely high or low calories.
    Unusually large or small numbers of ingredients.

In [16]:
def iqr_method(outlier):
    #Removing Outliers by IQR Method
    q1=df[outlier].quantile(0.25)
    q3=df[outlier].quantile(0.75)
    iqr=q3-q1
    upper_limit = q3 + (1.5*iqr)
    lower_limit = q1 - (1.5*iqr)
    
    df.loc[(df[outlier]>upper_limit),outlier]=upper_limit
    df.loc[(df[outlier]<lower_limit),outlier]=lower_limit
iqr_method('calories')
iqr_method('no_ingredients')
iqr_method('total_mins')

/tmp/ipykernel_22165/2791429900.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '933.375' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df[outlier]>upper_limit),outlier]=upper_limit
/tmp/ipykernel_22165/2791429900.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '312.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[(df[outlier]>upper_limit),outlier]=upper_limit


In [17]:
df_exploded = df.explode('season')
df_exploded2 = df.explode('meal_type')

**TOKENIZATION OF TITLE COLUMN**

REMOVING OF STOP WORDS

STOP WORDS - those words that occur very frequently in a statement that provide no particular meaning for the ML model training. it is simply existing for grammatical uses like connectors, verbs etc.

eg. is, am, the, where, when, that, this etc.

REMOVING SPECIAL CHARACTERS

punctuations and other decorators were also removed for simplicity and accurate prediction of model.

In [18]:
def clean_text(text):
    text = text.lower()
    nopunct_text = ''.join([char for char in text if char.isalnum() or char.isspace()])
    tokens = word_tokenize(nopunct_text)
    return tokens

df['cleaned_titles'] = df['title'].apply(clean_text)

In [19]:
lemmatizer = WordNetLemmatizer()

def lemmatize_feature(df, feature_name):
    df[feature_name] = df[feature_name].apply(lambda row: lemmatize_list(row))
    return df

def lemmatize_list(list_of_strings):
    lemmatized_list = [lemmatizer.lemmatize(word) for word in list_of_strings]
    return lemmatized_list

data = lemmatize_feature(df, 'cleaned_titles')

Pre - Processing the CATEGORY feature

removal of any special characters, conversion to lower cases and lemmatization of words for better computation
after lemmatization, the stop words were removed.

In [20]:
def lemmatize_category_list(category_list):
    lemmatized_list = []
    for word in category_list:
        lemmatized_category = []
        lemmatized_category.append(lemmatizer.lemmatize(word.strip().lower()))
        lemmatized_list.append(lemmatized_category)
    return lemmatized_list

df['category_lemmatized'] = df['category'].apply(lemmatize_category_list)

In [21]:
def preprocess_ingredients(ingredients):

    ingredients=[ingredient.lower() for ingredient in ingredients]

    pattern=r'[^\w]'  # Matches any non-alphanumeric character
    ingredients=[re.sub(pattern, ' ', ingredient) for ingredient in ingredients]

    lemmatizer=WordNetLemmatizer()
    processed_ingredients=[[lemmatizer.lemmatize(word) for word in ingredients]]

    return processed_ingredients

ingre=df['ingredients'].apply(preprocess_ingredients)

stop_words = stopwords.words('english') 

def filter_custom_stop_words(data):
    new_stop_words =['taste','softened','teaspoon','teaspoons','tablespoons','cups','ounces','tablespoon',
                     'cup','ounce','pinch','salt','ground','large','finely','chopped','grated','optional',
                     'crushed','drained','divided','shredded','sliced','half','peeled','freshly','needed',
                     'frying','prepared','cut','small','cube','water','warm','ml','l','g','kg','mg','dl',
                     'gill','diced','pound','pint','quant','gallon','fl oz','°C','°F']
    for i in new_stop_words:
        stop_words.append(i)
        
    filtered_data = []
    for recipe in data:
        sublist2 = []
        for sublist2_item in recipe:
            for i in sublist2_item:
                filtered_sublist3 = [word for word in i.split() if word not in stop_words and word.isalpha()]
                sublist2.append(filtered_sublist3)
        filtered_data.append(sublist2)
    return filtered_data

#function call for filtering
filtered_data = filter_custom_stop_words(ingre)

df['ingredients_filtered']=filtered_data

def join_lists(data):

    joined_list = []
    for sublist in data:
        joined_list.append([' '.join(x) for x in sublist])  # Join elements with space
    return joined_list

# Join elements in each sublist
df['ingredients_filtered'] = join_lists(df['ingredients_filtered'])
df['category_lemmatized'] = join_lists(df['category_lemmatized'])
df['category_lemmatized']=df['category_lemmatized'].apply(lambda x: ', '.join(x))

df.head()

,calories,category,ingredients,avg_rating,recipe_id,no_reviews,title,total_mins,season,meal_type,diet_type,no_ingredients,cleaned_titles,category_lemmatized,ingredients_filtered
0,728.0,"[Desserts, Fruit Desserts, Pineapple Desserts]","[2 cups all-purpose flour, 1 teaspoon salt, 1 ...",4.703704,263516,23,Chef John's Carrot Cake,95.0,"[Fall, Winter]",[Dessert],nonveg,18,"[chef, john, carrot, cake]","dessert, fruit desserts, pineapple desserts","[purpose flour, , baking soda, baking powder, ..."
1,692.0,"[Bread, Yeast Bread]","[1/2 cup warm milk, 1/3 cup warm water, 1 1/2 ...",4.666667,272625,2,Khachapuri (Georgian Cheese Bread),115.0,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",nonveg,14,"[khachapuri, georgian, cheese, bread]","bread, yeast bread","[milk, , white sugar, package active dry yeast..."
2,209.0,"[Soups, Stews and Chili, Stews, Beef]","[1 (1 1/2-pound) flank steak, 2 teaspoons kosh...",5.000000,272615,4,Ropa Vieja (Cuban Beef),312.5,"[Summer, Spring, Fall, Winter]",[Dinner],nonveg,23,"[ropa, vieja, cuban, beef]","soups, stews and chili, stew, beef","[flank steak, kosher, black pepper, cayenne pe..."
3,505.0,"[Everyday Cooking, Vegetarian, Side Dishes]","[2 cups whole milk, 1 teaspoon kosher salt, or...",4.333333,272461,9,"""Instant"" Mac and Cheese",28.0,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",veg,10,"[instant, mac, and, cheese]","everyday cooking, vegetarian, side dishes","[whole milk, kosher, cayenne pepper, dried mus..."
4,291.0,"[Bread, Yeast Bread]","[1 cup warm water, 1 tablespoon white sugar, 1...",5.000000,271049,4,Kouign-Amann,275.0,"[Summer, Spring, Fall, Winter]",[Dessert],nonveg,10,[kouignamann],"bread, yeast bread","[, white sugar, active dry yeast, bread flour,..."


In [22]:
df=df.drop(['title','category','ingredients'],axis=1)

In [23]:
for i in df['ingredients_filtered']:
    for j in i:
        if len(j)<1:
            i.remove(j)
            
def to_string(value):
    comb_str=' '.join(value)
    return comb_str

df['ingredients_filtered']=df['ingredients_filtered'].apply(to_string)
ingredients_filtered=df[['recipe_id','ingredients_filtered']]

In [24]:
df.head()

,calories,avg_rating,recipe_id,no_reviews,total_mins,season,meal_type,diet_type,no_ingredients,cleaned_titles,category_lemmatized,ingredients_filtered
0,728.0,4.703704,263516,23,95.0,"[Fall, Winter]",[Dessert],nonveg,18,"[chef, john, carrot, cake]","dessert, fruit desserts, pineapple desserts",purpose flour baking soda baking powder ginger...
1,692.0,4.666667,272625,2,115.0,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",nonveg,14,"[khachapuri, georgian, cheese, bread]","bread, yeast bread",milk white sugar package active dry yeast oliv...
2,209.0,5.000000,272615,4,312.5,"[Summer, Spring, Fall, Winter]",[Dinner],nonveg,23,"[ropa, vieja, cuban, beef]","soups, stews and chili, stew, beef",flank steak kosher black pepper cayenne pepper...
3,505.0,4.333333,272461,9,28.0,"[Summer, Spring, Fall, Winter]","[Lunch, Dinner]",veg,10,"[instant, mac, and, cheese]","everyday cooking, vegetarian, side dishes",whole milk kosher cayenne pepper dried mustard...
4,291.0,5.000000,271049,4,275.0,"[Summer, Spring, Fall, Winter]",[Dessert],nonveg,10,[kouignamann],"bread, yeast bread",white sugar active dry yeast bread flour melte...


**RECOMMENDATION BASED ON INGREDIENT SIMILARITY**

I DID THIS BY TAKING THE VECTOR FORMAT OF THE INGREDIENTS, CATEGORY AND TITLE COLUMN OF EACH RECIPE AND THEN FURTHER PLOTTING THE SIMILARITY BETWEEN EACH RECIPE USING COSINE SIMILARITY.
THIS IS DONE BY USING THE K-NEAREST NEIGHBOURS ALGORITHM. HERE K IS CHOSEN TO BE 10.


THIS MODEL SUGGESTS SIMILAR RECIPES WHEN ONE RECIPE IS VIEWED. THIS COULD GIVE OPTIONS FOR THE USER TO PICK.

In [25]:
vectorizer=TfidfVectorizer()

df['combined_text'] = df['cleaned_titles'].apply(' '.join) + ' ' + df['ingredients_filtered'] + ' ' + df['category_lemmatized']


recipe_text_features = vectorizer.fit_transform(df['combined_text'])

similarity=cosine_similarity(recipe_text_features, recipe_text_features)

similarity

array([[1.        , 0.11679077, 0.02704598, ..., 0.03257952, 0.00442233,
        0.03347495],
       [0.11679077, 1.        , 0.05286373, ..., 0.00880162, 0.03889843,
        0.21937197],
       [0.02704598, 0.05286373, 1.        , ..., 0.040295  , 0.11254314,
        0.06270797],
       ...,
       [0.03257952, 0.00880162, 0.040295  , ..., 1.        , 0.06596972,
        0.01784374],
       [0.00442233, 0.03889843, 0.11254314, ..., 0.06596972, 1.        ,
        0.04926132],
       [0.03347495, 0.21937197, 0.06270797, ..., 0.01784374, 0.04926132,
        1.        ]])

In [26]:
knn = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='cosine')
knn.fit(recipe_text_features)

def recommend_recipe(recipe_id):
    
    if recipe_id not in df['recipe_id'].values:
        print("Recipe ID not found.")
        return []
    
    recipe_index = df.index[df['recipe_id'] == recipe_id].tolist()[0]
    
    #fetching nearest neighbors for the recipe at the found index
    distances, indices = knn.kneighbors(recipe_text_features[recipe_index].reshape(1, -1), n_neighbors=10)
    recommendations = []

    for i in range(1, len(distances.flatten())):
        recommended_recipe_id = df['recipe_id'][indices.flatten()[i]]
        recommendations.append(recommended_recipe_id)
    
    return recommendations

#example
recommendations_list = recommend_recipe(220596)
print("Recommendations list:", recommendations_list)

Recommendations list: [263516, 263516, 222337, 263749, 221068, 246868, 221954, 232799, 254553]


In [27]:
pickle.dump(recommend_recipe, open('KNN_model.pkl','wb'))
loaded_recommend_recipes = pickle.load(open("KNN_model.pkl", "rb"))
print(loaded_recommend_recipes(222337))

[229442, 223041, 219636, 220596, 255011, 230318, 263516, 263516, 245442]


In [28]:
df['ingredients_filtered'] = df['ingredients_filtered'].apply(lambda x: ''.join(x))
df['cleaned_titles'] = df['cleaned_titles'].apply(lambda x: ' '.join(x))

**MATRIX FACTORIZATION USING USER RATING AND COMBINATION OF INGREDIENTS,TITLE AND CATEGORY OF FOOD**

MATRIX FACTORIZATION IS COMMONLY USED FOR DETECTING A PATTERN BETWEEN DIFFERENT USERS AND THEIR INTEREST. SINCE WE HAVE DATASET OF USER RATING FOR DIFFERENT RECIPES, WE COULD PLOT USER VS. RECIPE MATRIX. THIS COULD SUGGEST RECIPES FOR A USER THAT WAS ACCEPTED BY A SIMILAR USER.

FOR THIS MODEL, THE INPUT IS USER_ID AND OUTPUT IS A SERIES OF RECIPE_ID'S


In [29]:
user_recipe_df=df2.merge(df,on='recipe_id')

#user-recipe matrix

user_recipe_df = user_recipe_df[user_recipe_df['rating'] >= 3]
rating_matrix=user_recipe_df.pivot_table(values='rating',index='user_id',columns='recipe_id').fillna(0)
rating_matrix

recipe_id,219075,219076,219077,219078,219079,219082,219086,219088,219090,219091,...,272615,272625,272766,272767,272905,273034,273213,273285,273455,273495
user_id,,,,,,,,,,,,,,,,,,,,,
16,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
935,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25586309,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25591130,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25594700,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
num_factors=10

#perform NMF Non-Negative Matrix Factorization
nmf = NMF(n_components=num_factors, random_state=42)
user_factors=nmf.fit_transform(rating_matrix)
recipe_factors=nmf.components_

user_id_to_index = {user_id: idx for idx, user_id in enumerate(rating_matrix.index)}

# Function to recommend recipes based on user ID
def recommend_recipes(user_id, top_n=10):
    if user_id not in user_id_to_index:
        print(f"User ID {user_id} not found.")
        return []
    
    user_index = user_id_to_index[user_id]
    user_factor = user_factors[user_index]
    predicted_ratings = user_factor.dot(recipe_factors)
    top_recipe_indices = predicted_ratings.argsort()[-top_n:][::-1]  # Get top n indices, sorted in descending order
    
    # Map indices back to recipe IDs
    top_recipe_ids = rating_matrix.columns[top_recipe_indices]
    return top_recipe_ids

#recommendations for user with ID
recommended_recipes_CF=recommend_recipes(16)

print(recommended_recipes_CF)

In [ ]:
pickle.dump(recommend_recipes, open('CF_model.pkl','wb'))
CF_recommend_recipes = pickle.load(open("CF_model.pkl", "rb"))
CF_recommend_recipes(16)